In [1]:
import pickle

In [2]:
PATH='/home/kirana/Documents/phd/experiment1_new/DNNREC_BookCrossing'
DATAPATH='/home/kirana/Documents/final_dissertation_final/experiments/datasets/bookcrossing'

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sklearn
import os
import pickle
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import pickle
from sklearn import metrics

In [4]:
[dftrain,dfvalid]=pickle.load(open(f'{DATAPATH}/df_side.pkl','rb'))

In [5]:
trainusers=set(dftrain['userId'].unique())
validusers=set(dfvalid['userId'].unique())
trainitems=set(dftrain['itemId'].unique())
validitems=set(dfvalid['itemId'].unique())

In [6]:
commonusers=trainusers.intersection(validusers)
commonitems=trainitems.intersection(validitems)

In [7]:
dfvalid['cold_start_user']=~dfvalid['userId'].isin(commonusers)
dfvalid['cold_start_item']=~dfvalid['itemId'].isin(commonitems)

In [8]:
print (pd.crosstab(dfvalid['cold_start_user'],dfvalid['cold_start_item']))
print (pd.crosstab(dfvalid['cold_start_user'],dfvalid['cold_start_item'])/dfvalid.shape[0])   

cold_start_item  False  True 
cold_start_user              
False            24016  10932
True              5492   2232
cold_start_item     False     True 
cold_start_user                    
False            0.562805  0.256187
True             0.128703  0.052306


In [9]:
dftrain['userId'].value_counts()

11676     7705
98391     5223
153662    1969
189835    1711
23902     1256
235105     935
171118     925
76499      916
248718     875
16795      872
56399      825
35859      750
63714      740
197659     721
185233     649
95359      564
114368     548
101851     541
158295     512
177458     512
76626      509
23872      508
135149     493
78973      486
257204     476
69078      471
93047      469
3757       464
204864     464
182085     457
          ... 
45954        1
275218       1
203511       1
52365        1
27801        1
191945       1
267542       1
169414       1
73143        1
118189       1
116140       1
128426       1
26008        1
5526         1
38278        1
46466        1
42368        1
224505       1
113827       1
206064       1
253167       1
247020       1
152798       1
150749       1
165060       1
81075        1
128170       1
124072       1
101541       1
12294        1
Name: userId, Length: 72887, dtype: int64

In [10]:
dfvalid['cold_start_group']=np.where(dfvalid['cold_start_user'] & dfvalid['cold_start_item'],'User-Item-cold-start',
                                    np.where((~dfvalid['cold_start_user']) & (~dfvalid['cold_start_item']), 'No-cold-start',\
                                            np.where(dfvalid['cold_start_user'], 'User-Cold-Start', 'Item-Cold_start')))    


In [11]:
dfvalid['cold_start_bucket']=np.where(dfvalid['cold_start_group']=='No-cold-start',0,1)

In [12]:
dfvalid['cold_start_group'].value_counts()

No-cold-start           24016
Item-Cold_start         10932
User-Cold-Start          5492
User-Item-cold-start     2232
Name: cold_start_group, dtype: int64

In [13]:
dfvalid['cold_start_bucket'].value_counts()

0    24016
1    18656
Name: cold_start_bucket, dtype: int64

In [14]:
pickle.dump([dftrain,dfvalid],open(f'{DATAPATH}/df_side_cold.pkl','wb'))

In [15]:
dftrain.shape,dfvalid.shape

((390999, 13), (42672, 17))